# Read Inspection data


In [11]:
import pandas as pd
import os.path
import re
path = 'D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/01 - Inspections/June 2019/'

In [53]:
file_list = []
for root, dirs, files in os.walk(path):
    for filename in files:
        if filename.endswith('.csv'):
            file_list.append(os.path.join(root, filename)) 
    df_list = [pd.read_csv(file, index_col = 'Survey ID', usecols = range(0, 231), nrows=1) for file in file_list]
    df_list_bd = [pd.read_csv(file, usecols = range(0, 244)) for file in file_list]
if df_list:
    inspection_data = pd.concat(df_list, axis=0, sort=False) 
    bd_data = pd.concat(df_list_bd, axis=0, sort=False) 

In [54]:
bd_data.head()

,Survey ID,Survey Name,Section Name,Date and Time of Survey,Vehicle Class - Standard bus,Vehicle Class - Minibus,Vehicle Class - Midibus,Section Name.1,Interior Inspection,Odometer Reading,...,Canvas Image,Section Name.7,General Information,Marker Name,Marker Id,Last Modified By,Last Modified At,Section Name.8,Nature of Damage,When does the vehicles roadworthy expire?
0,2e714d30-3d05-1197-4d52-9bc9d1f5d015,Bus 100,General,"Tuesday, May 28, 2019 12:34:00 PM",Standard bus,,,General Interior,NaN,NaN,...,1f64c3e1-abb3-b18f-ddb3-adb063ab4123new.jpg,NaN,NaN,Bus 100.4,4e32eec5-3503-4790-b3d1-54f20304a8b2,Katlego.mashishi@aurecongroup.com,6/4/2019 10:42:42 AM,External Damage,Body Damage,NaN
1,2e714d30-3d05-1197-4d52-9bc9d1f5d015,Bus 100,General,"Tuesday, May 28, 2019 12:34:00 PM",Standard bus,,,General Interior,NaN,NaN,...,NaN,NaN,NaN,Bus 100.1,8e29fdd8-f664-4b58-ba49-7eece72681bd,Katlego.mashishi@aurecongroup.com,6/4/2019 10:42:21 AM,External Damage,Body Damage,NaN
2,2e714d30-3d05-1197-4d52-9bc9d1f5d015,Bus 100,General,"Tuesday, May 28, 2019 12:34:00 PM",Standard bus,,,General Interior,NaN,NaN,...,NaN,NaN,NaN,Bus 100.2,d1a1c45f-5cc0-4362-a783-f2e20d87b088,Katlego.mashishi@aurecongroup.com,6/4/2019 10:42:28 AM,External Damage,Body Damage,NaN
3,2e714d30-3d05-1197-4d52-9bc9d1f5d015,Bus 100,General,"Tuesday, May 28, 2019 12:34:00 PM",Standard bus,,,General Interior,NaN,NaN,...,NaN,NaN,NaN,Bus 100.3,e5d58ce9-cca9-47a9-9313-f62695a4b2b7,Katlego.mashishi@aurecongroup.com,6/4/2019 10:42:35 AM,External Damage,Body Damage,NaN
0,ecb39b86-3657-48c5-1b63-999eff72dc6b,Bus 101,General,"Monday, June 03, 2019 2:46:00 PM",Standard bus,,,General Interior,NaN,NaN,...,b0b71f2f-ce7c-9470-af6b-65c8d9a0b885new.jpg,NaN,NaN,Bus 101.4,6caf4855-e9cd-4938-a37e-308f9d545bcf,Katlego.mashishi@aurecongroup.com,6/4/2019 1:18:52 PM,External Damage,Body Damage,NaN


## Adjust column names, filter and restructure inspection data

In [49]:
insdf = inspection_data.drop(['Section Name', 
                            'Section Name.1', 'Exterior Inspection',
                            'Section Name.2', 'Interior Inspection', 
                            'Section Name.3', 'Section Name.4', 
                            'Section Name.5', 'Section Name.6',
                            'When does the vehicles roadworthy expire?'], axis=1)

insdf['Vehicle Class (Value)'] = insdf['Vehicle Class - Standard bus'].map(str) + insdf['Vehicle Class - Minibus'].map(str) +insdf['Vehicle Class - Midibus'].map(str)

insdf = insdf.drop(['Vehicle Class - Standard bus',
                    'Vehicle Class - Minibus',
                    'Vehicle Class - Midibus'], axis=1)

insdf = insdf[insdf.columns.drop(list(insdf.filter(regex=' - No')))]
insdf = insdf[insdf.columns.drop(list(insdf.filter(regex='Image')))]
insdf = insdf[insdf.columns.drop(list(insdf.filter(regex='Photo')))]
insdf = insdf[insdf.columns.drop(list(insdf.filter(regex='Details')))]
insdf = insdf[insdf.columns.drop(list(insdf.filter(regex='details')))]
insdf = insdf[insdf.columns.drop(list(insdf.filter(regex='Date by')))]

insdf.columns = insdf.columns.str.replace("Are ", "")
insdf.columns = insdf.columns.str.replace("there ", "")
insdf.columns = insdf.columns.str.replace("any  ", "")
insdf.columns = insdf.columns.str.replace("Is ", "")
insdf.columns = insdf.columns.str.replace("the ", "")
insdf.columns = insdf.columns.str.replace("any ", "")
insdf.columns = insdf.columns.str.replace(".1", "")
insdf.columns = insdf.columns.str.replace(" - Yes", "")
insdf.columns = insdf.columns.str.replace("?", "")

insdefects = pd.melt(insdf, id_vars=['Survey Name', 'Date and Time of Survey',
   'Vehicle Class (Value)', 'Odometer Reading'], var_name= 'Defect', value_name ='Present')

insdefects = insdefects.fillna('')

insdefects_filtered = insdefects[insdefects['Present'] == 'Yes']

In [50]:
insdefects_filtered.columns

Index(['Survey Name', 'Date and Time of Survey', 'Vehicle Class (Value)',
       'Odometer Reading', 'Defect', 'Present'],
      dtype='object')

## Restructure Body Damage data


In [55]:
bd_data['Vehicle Class (Value)'] = bd_data['Vehicle Class - Standard bus'].map(str) + bd_data['Vehicle Class - Minibus'].map(str) + bd_data['Vehicle Class - Midibus'].map(str)

bd_filtered = bd_data[pd.notnull(bd_data['Nature of Damage'])]
bd_filled = bd_filtered.fillna(method='ffill')
bd_filled = bd_filled.set_index('Survey ID')
bd_filled = bd_filled.filter(items=['Survey Name', 
                                    'Date and Time of Survey', 
                                    'Vehicle Class (Value)', 
                                    'Last Modified By', 
                                    'Last Modified At', 
                                    'Nature of Damage'])
print(bd_filled.shape)
bd_filled.columns

(202, 6)


Index(['Survey Name', 'Date and Time of Survey', 'Vehicle Class (Value)',
       'Last Modified By', 'Last Modified At', 'Nature of Damage'],
      dtype='object')

## Save data


In [58]:
path = 'D:/001_Projects/01 - GIPTN/02 - Application development/03 - Bus Inspection/01 - Inspections/June 2019/Results/'

filename = 'Full List.csv'
insdefects.to_csv(path + filename)
filename = 'Defect List2.csv'
insdefects_filtered.to_csv(path + filename)
filename = 'Body Damage2.csv'
bd_filled.to_csv(path + filename)